# Advent of Code 2017: [Day 10](http://adventofcode.com/2017/day/10)

## Problem statement

>You come across some programs that are trying to implement a software emulation of a hash based on knot-tying. The hash these programs are implementing isn't very strong, but you decide to help them anyway. You make a mental note to remind the Elves later not to invent their own cryptographic functions.

>This hash function simulates tying a knot in a circle of string with 256 marks on it. Based on the input to be hashed, the function repeatedly selects a span of string, brings the ends together, and gives the span a half-twist to reverse the order of the marks within it. After doing this many times, the order of the marks is used to build the resulting hash.

>  4--5   pinch   4  5           4   1

> /    \  5,0,1  / \/ \  twist  / \ / \

>3      0  -->  3      0  -->  3   X   0

> \    /         \ /\ /         \ / \ /

>  2--1           2  1           2   5

>To achieve this, begin with a list of numbers from 0 to 255, a current position which begins at 0 (the first element in the list), a skip size (which starts at 0), and a sequence of lengths (your puzzle input).  Then, for each length:

>Reverse the order of that length of elements in the list, starting with the element at the current position.

>Move the current position forward by that length plus the skip size.

>Increase the skip size by one.

>The list is circular; if the current position and the length try to reverse elements beyond the end of the list, the operation reverses using as many extra elements as it needs from the front of the list. If the current position moves past the end of the list, it wraps around to the front. Lengths larger than the size of the list are invalid.

>Here's an example using a smaller list:

>Suppose we instead only had a circular list containing five elements, 0, 1, 2, 3, 4, and were given input lengths of 3, 4, 1, 5.

>The list begins as [0] 1 2 3 4 (where square brackets indicate the current position).

>The first length, 3, selects ([0] 1 2) 3 4 (where parentheses indicate the sublist to be reversed).

>After reversing that section (0 1 2 into 2 1 0), we get ([2] 1 0) 3 4.

>Then, the current position moves forward by the length, 3, plus the skip size, 0: 2 1 0 [3] 4. Finally, the skip size increases to 1.

>The second length, 4, selects a section which wraps: 2 1) 0 ([3] 4.

>The sublist 3 4 2 1 is reversed to form 1 2 4 3: 4 3) 0 ([1] 2.

>The current position moves forward by the length plus the skip size, a total of 5, causing it not to move because it wraps around: 4 3 0 [1] 2. The skip size increases to 2.

>The third length, 1, selects a sublist of a single element, and so reversing it has no effect.

>The current position moves forward by the length (1) plus the skip size (2): 4 [3] 0 1 2. The skip size increases to 3.

>The fourth length, 5, selects every element starting with the second: 4) ([3] 0 1 2. Reversing this sublist (3 0 1 2 4 into 4 2 1 0 3) produces: 3) ([4] 2 1 0.

>Finally, the current position moves forward by 8: 3 4 2 1 [0]. The skip size increases to 4.

>In this example, the first two numbers in the list end up being 3 and 4; to check the process, you can multiply them together to produce 12.

>However, you should instead use the standard list size of 256 (with values 0 to 255) and the sequence of lengths in your puzzle input. Once this process is complete, what is the result of multiplying the first two numbers in the list?

## Breaking down the problem
- **Task**:
- <font color='green'>Input</font>:
- <font color='blue'>Process the data</font>:
- <font color='red'>Compute</font>:

## Implementation

In [4]:
from itertools import islice

def reverse(sequence, start, length):
    indices = [i % len(sequence) for i in range(start, start + length)]
    pairs = zip(indices, reversed(indices))

    for i, j in islice(pairs, length // 2):
        sequence[j], sequence[i] = sequence[i], sequence[j]

In [5]:
def twists(circle, lengths, position=0, skip_size=0):
    while True:
        for length in lengths:
            reverse(circle, position, length)
            position += length + skip_size
            skip_size += 1
        yield

## Check against test cases

In [6]:
circle = list(range(5))
lengths = [3, 4, 1, 5]

next(twists(circle, lengths))

print(circle[0] * circle[1])

12


## Solve problem

In [7]:
circle = list(range(256))
lengths = [106,118,236,1,130,0,235,254,59,205,2,87,129,25,255,118]

next(twists(circle, lengths))
    
print(circle[0] * circle[1])

6909


# Part 2

In [8]:
from functools import reduce
from operator import xor

def blocks(sequence, length=16):
    for i in range(0, len(sequence), length):
        yield sequence[i:(i + length)]
        
def knot_hash(string):
    circle = list(range(256))
    lengths = [ord(digit) for digit in string] + [17, 31, 73, 47, 23]

    rounds = twists(circle, lengths)
    for _ in range(64): next(rounds)

    sparse_hash = circle
    dense_hash = [reduce(xor, block) for block in blocks(sparse_hash, 16)]
    return ''.join(format(number, '02x') for number in dense_hash)

## Check against test cases

In [9]:
assert knot_hash('')         == 'a2582a3a0e66e6e86e3812dcb672a272'
assert knot_hash('AoC 2017') == '33efeb34ea91902bb2f59c9920caa6cd'
assert knot_hash('1,2,3')    == '3efbe78a8d82f29979031a4aa0b16a9d'
assert knot_hash('1,2,4')    == '63960835bcdc130f0b66d7ff4f6a5a8e'

In [10]:
circle = list(range(256))
string = '106,118,236,1,130,0,235,254,59,205,2,87,129,25,255,118'

print(knot_hash(string))

9d5f4561367d379cfbf04f8c471c0095
